# Call Variants in Juicer Data to Verify Correct Samples Used
- **Author** - Frank Grenn
- **Date Started** - March 2020
- **Quick Description:** remap the HiC fastqs to get bams. Then subset by the loop region bedpes created by juicer. Then call variants in those subsetted bams using bcftools

In [ ]:
import os
import pandas as pd

## 1) Setup

In [ ]:
#these paths and files need to already exist
FASTQDIR = "/path/to/temp_fastqs" #location of the combined lane fastqs
JUICERDIR = "/path/to/juicer" #location of the output from juicer
WRKDIR="/path/tp/callVariants" #folder that will contain a folder for each sample's HiC-Pro output
REF_GENOME_FASTA="/path/to/Homo_sapiens_assembly38.fasta" #pulled down from gs://broad-references/hg38/v0/Homo_sapiens_assembly38.fasta



In [ ]:
!mkdir {WRKDIR}

Setup directories per sample

In [ ]:
sample_directories = [ name for name in os.listdir(FASTQDIR) if os.path.isdir(os.path.join(FASTQDIR, name)) and "HICS" in name]
print(len(sample_directories))
print(sample_directories)

In [ ]:
for sample in sample_directories:
    !mkdir {WRKDIR}/{sample}
    !mkdir {WRKDIR}/{sample}/fastq
    !ln -s {FASTQDIR}/{sample}/{sample}_R1_001.fastq.gz {WRKDIR}/{sample}/fastq
    !ln -s {FASTQDIR}/{sample}/{sample}_R2_001.fastq.gz {WRKDIR}/{sample}/fastq
    

In [ ]:
with open(WRKDIR+"/map_samples.swarm","w") as swarm_file:
    for sample in sample_directories:
        with open(WRKDIR+"/"+sample+"/map_and_subset.sh", "w") as sample_file:
            sample_file.write(f"#!/bin/bash \n\
            module load samtools \n\
            module load bwa \n\
            echo 'aligning' \n\
            bwa mem -t 20 {REF_GENOME_FASTA} {WRKDIR}/{sample}/fastq/{sample}_R1_001.fastq.gz {WRKDIR}/{sample}/fastq/{sample}_R2_001.fastq.gz > {WRKDIR}/{sample}/{sample}_001.sam \n\
            echo 'finished aligning' \n\
            echo 'convert sam to bam' \n\
            samtools view -@20 -b {WRKDIR}/{sample}/{sample}_001.sam -o {WRKDIR}/{sample}/{sample}_001.bam \n\
            echo 'finished converting sam to bam' \n\
            rm {WRKDIR}/{sample}/{sample}_001.sam \n\
            echo 'sort bam' \n\
            samtools sort -@20 {WRKDIR}/{sample}/{sample}_001.bam -o {WRKDIR}/{sample}/{sample}_001.sort.bam \n\
            echo 'finished sorting bam' \n\
            awk '{{print $1,$2,$3}}' {JUICERDIR}/{sample}/aligned/inter_30_loops/merged_loops_nh.bedpe > {WRKDIR}/{sample}/loop_anchor1.bed \n\
            awk '{{print $4,$5,$6}}' {JUICERDIR}/{sample}/aligned/inter_30_loops/merged_loops_nh.bedpe > {WRKDIR}/{sample}/loop_anchor2.bed \n\
            cat {WRKDIR}/{sample}/loop_anchor1.bed {WRKDIR}/{sample}/loop_anchor2.bed >> {WRKDIR}/{sample}/loop_anchors.bed \n\
            echo 'subset bam by region' \n\
            samtools view -@20 {WRKDIR}/{sample}/{sample}_001.sort.bam -L {WRKDIR}/{sample}/loop_anchors.bed -b -o {WRKDIR}/{sample}/{sample}_001.loopsub.bam \n\
            echo 'finished subsetting bam by regions' \n\
            echo 'sort subset bam' \n\
            samtools sort -@20 {WRKDIR}/{sample}/{sample}_001.loopsub.bam -o {WRKDIR}/{sample}/{sample}_001.loopsub.sort.bam \n\
            echo 'finished sort subset bam' \n\
            echo 'indexing' \n\
            samtools index -@20 {WRKDIR}/{sample}/{sample}_001.loopsub.sort.bam \n\
            echo 'done' \n\
            ")
        sample_file.close()
        
        
        
        swarm_file.write(f"bash {WRKDIR}/{sample}/map_and_subset.sh \n")
swarm_file.close()
                         

In [ ]:
print(f"swarm -f {WRKDIR}/map_samples.swarm -g 100 --time=24:00:00 -t 20 --sbatch '--mail-type=ALL'")

#### if the jobs timed out use what was generated
remove last line from the sam file if alignment didn't finish

In [ ]:
samples_to_finish = ['HICS_PPMI51971_9029_da65_v1_S8','HICS_PPMI3666_3014_da65_v1_S4']

In [ ]:
with open(WRKDIR+"/map_unfinished_samples.swarm","w") as swarm_file:
    for sample in samples_to_finish:
        with open(WRKDIR+"/"+sample+"/map_and_subset.sh", "w") as sample_file:
            sample_file.write(f"#!/bin/bash \n\
            module load samtools \n\
            module load bwa \n\
            echo 'remove last line from sam'\n\
            mv {WRKDIR}/{sample}/{sample}_001.sam {WRKDIR}/{sample}/{sample}_001_part.sam\n\
            sed '$d' {WRKDIR}/{sample}/{sample}_001_part.sam > {WRKDIR}/{sample}/{sample}_001.sam\n\
            echo 'convert sam to bam' \n\
            samtools view -@20 -b {WRKDIR}/{sample}/{sample}_001.sam -o {WRKDIR}/{sample}/{sample}_001.bam \n\
            echo 'finished converting sam to bam' \n\
            rm {WRKDIR}/{sample}/{sample}_001.sam \n\
            echo 'sort bam' \n\
            samtools sort -@20 {WRKDIR}/{sample}/{sample}_001.bam -o {WRKDIR}/{sample}/{sample}_001.sort.bam \n\
            echo 'finished sorting bam' \n\
            awk '{{print $1,$2,$3}}' {JUICERDIR}/{sample}/aligned/inter_30_loops/merged_loops_nh.bedpe > {WRKDIR}/{sample}/loop_anchor1.bed \n\
            awk '{{print $4,$5,$6}}' {JUICERDIR}/{sample}/aligned/inter_30_loops/merged_loops_nh.bedpe > {WRKDIR}/{sample}/loop_anchor2.bed \n\
            cat {WRKDIR}/{sample}/loop_anchor1.bed {WRKDIR}/{sample}/loop_anchor2.bed >> {WRKDIR}/{sample}/loop_anchors.bed \n\
            echo 'subset bam by region' \n\
            samtools view -@20 {WRKDIR}/{sample}/{sample}_001.sort.bam -L {WRKDIR}/{sample}/loop_anchors.bed -b -o {WRKDIR}/{sample}/{sample}_001.loopsub.bam \n\
            echo 'finished subsetting bam by regions' \n\
            echo 'sort subset bam' \n\
            samtools sort -@20 {WRKDIR}/{sample}/{sample}_001.loopsub.bam -o {WRKDIR}/{sample}/{sample}_001.loopsub.sort.bam \n\
            echo 'finished sort subset bam' \n\
            echo 'indexing' \n\
            samtools index -@20 {WRKDIR}/{sample}/{sample}_001.loopsub.sort.bam \n\
            echo 'done' \n\
            ")
        sample_file.close()
        
        
        
        swarm_file.write(f"bash {WRKDIR}/{sample}/map_and_subset.sh \n")
swarm_file.close()
                         

In [ ]:
print(f"swarm -f {WRKDIR}/map_unfinished_samples.swarm -g 100 --time=24:00:00 -t 20 --sbatch '--mail-type=ALL'")

# Call Variants

get samples that finished the mapping and subsetting

In [ ]:
finished_samples = [ name for name in os.listdir(WRKDIR) if os.path.isdir(os.path.join(FASTQDIR, name)) and os.path.exists(f"{WRKDIR}/{name}/{name}_001.loopsub.sort.bam")]


print(len(finished_samples))
print(finished_samples)

In [ ]:
with open(WRKDIR+"/call_sample_variants.swarm","w") as swarm_file:
    for sample in finished_samples:
        with open(WRKDIR+"/"+sample+"/call_variants.sh", "w") as sample_file:
            sample_file.write(f"#!/bin/bash \n\
            module load bcftools \n\
            echo 'making bcf' \n\
            bcftools mpileup --threads 20 -Ou -a 'FORMAT/AD' -a 'FORMAT/DP' -f /path/to/reference_genome/Homo_sapiens_assembly38.fasta /path/to/callVariants/{sample}/{sample}_001.loopsub.sort.bam | bcftools call -mv -Ob -o /path/to/callVariants/{sample}/{sample}.bcf\n\
            echo 'filtering to vcf'\n\
            bcftools view -V indels -i '%QUAL>=20 & INFO/DP>100' /path/to/callVariants/{sample}/{sample}.bcf > /path/to/callVariants/{sample}/{sample}_filtered.vcf\n\
            bcftools view {WRKDIR}/{sample}/{sample}_filtered.vcf -Oz -o {WRKDIR}/{sample}/{sample}_filtered.vcf.gz\n\
            bcftools index {WRKDIR}/{sample}/{sample}_filtered.vcf.gz\n\
            ")
        sample_file.close()
        
        
        
        swarm_file.write(f"bash {WRKDIR}/{sample}/call_variants.sh \n")
swarm_file.close()
                         

In [ ]:
print(f"swarm -f {WRKDIR}/call_sample_variants.swarm -g 100 --time=24:00:00 -t 20 --sbatch '--mail-type=ALL'")

# Merge Variants

In [ ]:
sample_vcfs = ' '.join([f"/path/to/callVariants/{name}/{name}_filtered.vcf.gz" for name in finished_samples])
print(sample_vcfs)

In [ ]:
print(f"bcftools merge {sample_vcfs} -o {WRKDIR}/merged_samples.vcf -Ov")

In [ ]:
print("plink --const-fid 0 --vcf merged_samples.vcf --make-bed --keep-allele-order --out plink")

format the .bim file to have chr:bp in the second col

In [ ]:
bim = pd.read_csv(f"{WRKDIR}/plink.bim",sep='\t',header = None)
bim.columns = ['chr','id','pos','bp','a1','a2']
print(bim.shape)
print(bim.head())

In [ ]:
bim[['id']] = "chr"+bim['chr'].astype(str)+":"+bim['bp'].astype(str)
print(bim.head())
print(bim.tail())

In [ ]:
bim.to_csv(f"{WRKDIR}/plink.bim",sep='\t',header=None,index=None)

## Compare to WGS Data

reformat the WGS .bim id (second) col to be chr:bp

In [ ]:
print(f'''awk '{{printf "%s\\tchr%s:%s\\t%s\\t%s\\t%s\\t%s\\n",$1,$1,$4,$3,$4,$5,$6}}' /path/to/wgshg38ppmi.july2018.bim > {WRKDIR}/wgshg38ppmi.july2018.bim''')
print(f"cp /path/to/wgshg38ppmi.july2018.bed {WRKDIR}")
print(f"cp /path/to/wgshg38ppmi.july2018.fam {WRKDIR}")

In [ ]:
print(f"plink --bfile {WRKDIR}/plink --bmerge {WRKDIR}/wgshg38ppmi.july2018 \
--maf 0.05 --geno 0.05 --hwe 1E-6 --make-bed --out {WRKDIR}/temp2")

In [ ]:
print(f"plink --bfile {WRKDIR}/plink --exclude temp2-merge.missnp --make-bed --out {WRKDIR}/HICS_plink_short")

In [ ]:
print(f"plink --bfile {WRKDIR}/HICS_plink_short --bmerge {WRKDIR}/wgshg38ppmi.july2018 \
--maf 0.05 --geno 0.05 --hwe 1E-6 --make-bed --out {WRKDIR}/temp2")

check relatedness

In [ ]:
print(f"plink --bfile {WRKDIR}/temp2 --indep-pairwise 500 5 0.5 --out {WRKDIR}/prune")

In [ ]:
print(f"plink --bfile {WRKDIR}/temp2 --extract prune.prune.in --make-bed --out {WRKDIR}/prune")

In [ ]:
print(f"plink --bfile {WRKDIR}/prune --genome --out {WRKDIR}/HICS_plink_relatedness_with_WGS --min 0.05")

In [ ]:
#reformat bc it seems off
print("awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13}' HICS_plink_relatedness_with_WGS.genome > HICS_plink_relatedness_with_WGS.format.genome")

In [ ]:
results = pd.read_csv(f"{WRKDIR}/HICS_plink_relatedness_with_WGS.format.genome",sep='\s')
print(results.shape)
print(results.head())
results.to_csv(f"{WRKDIR}/HICS_plink_relatedness_with_WGS.genome.csv",index=None)